<a href="https://colab.research.google.com/github/DmitriySutyagin/Introduction-to-neural-networks/blob/main/Home_work_Lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import re
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical

In [29]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [30]:
num_characters = 34 #33 буквы + пробел


In [31]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [32]:
tokenizer.fit_on_texts(text)

In [33]:
tokenizer.word_index

{' ': 1,
 'о': 2,
 'т': 3,
 'е': 4,
 'и': 5,
 'в': 6,
 'н': 7,
 'с': 8,
 'л': 9,
 'п': 10,
 'ь': 11,
 'ы': 12,
 'р': 13,
 'а': 14,
 'д': 15,
 'у': 16,
 'к': 17,
 'з': 18,
 'ч': 19,
 'й': 20,
 'м': 21,
 'г': 22,
 'б': 23,
 'я': 24,
 'ш': 25,
 'ю': 26,
 'х': 27}

In [34]:
inp_chars = 3
data = tokenizer.texts_to_matrix(text)
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
data[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
n = data.shape[0] - inp_chars
n

202

In [37]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [38]:
X[0]


array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [39]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 500)               267500    
                                                                 
 dense_2 (Dense)             (None, 34)                17034     
                                                                 
Total params: 284534 (1.09 MB)
Trainable params: 284534 (1.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 [==============================] - 1s 6ms/step - loss: 3.4358 - accuracy: 0.0941
Epoch 2/100
7/7 [==============================] - 0s 5ms/step - loss: 2.8139 - accuracy: 0.2970
Epoch 3/100
7/7 [==============================] - 0s 5ms/step - loss: 2.3704 - accuracy: 0.2970
Epoch 4/100
7/7 [==============================] - 0s 5ms/step - loss: 2.1132 - accuracy: 0.3762
Epoch 5/100
7/7 [==============================] - 0s 5ms/step - loss: 1.9092 - accuracy: 0.4455
Epoch 6/100
7/7 [==============================] - 0s 5ms/step - loss: 1.7413 - accuracy: 0.4802
Epoch 7/100
7/7 [==============================] - 0s 6ms/step - loss: 1.5968 - accuracy: 0.5198
Epoch 8/100
7/7 [==============================] - 0s 5ms/step - loss: 1.4696 - accuracy: 0.5446
Epoch 9/100
7/7 [==============================] - 0s 5ms/step - loss: 1.3805 - accuracy: 0.5347
Epoch 10/100
7/7 [==============================] - 0s 6ms/step - loss: 1.3025 - accuracy: 0.5891
Epoch 11/100
7/7 [===========

In [41]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [42]:
res = buildPhrase("вер")
res


1/1 [==============================] - 0s 24ms/step


'верьте понедельникли в понедельникли в понедельникли '

# ***Слова***

In [43]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ

In [44]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([text])

In [45]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [46]:
data = tokenizer.texts_to_sequences([text])
print(data, end = ' ')

[[1, 4, 5, 6, 7, 8, 9, 2, 3, 10, 11, 12, 13, 2, 14, 15, 16, 17, 18, 19, 1, 20, 2, 3, 21, 22, 23, 24, 1, 25, 26]] 

In [47]:
tokenizer.word_index

{'вы': 1,
 'в': 2,
 'понедельник': 3,
 'лучший': 4,
 'ответ': 5,
 'на': 6,
 'проблемы': 7,
 'которые': 8,
 'возникли': 9,
 'думайте': 10,
 'позитивно': 11,
 'и': 12,
 'верьте': 13,
 'свою': 14,
 'способность': 15,
 'достигать': 16,
 'отличных': 17,
 'результатов': 18,
 'если': 19,
 'смогли': 20,
 'подняться': 21,
 'с': 22,
 'постели': 23,
 'значит': 24,
 'супер': 25,
 'герой': 26}

In [48]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
res.shape

(31, 1000)

In [49]:
inp_words = 3
n = res.shape[0]-inp_words
res.shape

(31, 1000)

In [50]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [51]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_3 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
history = model.fit(X, Y, batch_size=32, epochs=19)

Epoch 1/19
1/1 [==============================] - 1s 821ms/step - loss: 6.9119 - accuracy: 0.0000e+00
Epoch 2/19
1/1 [==============================] - 0s 9ms/step - loss: 6.8858 - accuracy: 0.0000e+00
Epoch 3/19
1/1 [==============================] - 0s 10ms/step - loss: 6.8596 - accuracy: 0.0357
Epoch 4/19
1/1 [==============================] - 0s 10ms/step - loss: 6.8331 - accuracy: 0.1429
Epoch 5/19
1/1 [==============================] - 0s 10ms/step - loss: 6.8058 - accuracy: 0.3929
Epoch 6/19
1/1 [==============================] - 0s 9ms/step - loss: 6.7776 - accuracy: 0.5357
Epoch 7/19
1/1 [==============================] - 0s 9ms/step - loss: 6.7481 - accuracy: 0.7857
Epoch 8/19
1/1 [==============================] - 0s 11ms/step - loss: 6.7171 - accuracy: 0.9286
Epoch 9/19
1/1 [==============================] - 0s 9ms/step - loss: 6.6842 - accuracy: 0.9643
Epoch 10/19
1/1 [==============================] - 0s 9ms/step - loss: 6.6491 - accuracy: 0.9643
Epoch 11/19
1/1 [========

In [53]:
def buildPhrase_2(texts, str_len = 20):
  res =texts
  maxWordsCount =1000
  data = tokenizer.texts_to_sequences([texts])[0]
  data_reshaped = np.expand_dims(data, axis=0)
  data_batch = np.expand_dims(data_reshaped, axis=-1)

  for i in range(str_len):
      x = keras.utils.to_categorical(data_batch[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding

  inp = np.resize(x, (1, inp_words, maxWordsCount))


  pred = model.predict( inp ) # предсказываем OHE четвертого символа
  indx = pred.argmax(axis=1)[0]
  data.append(indx)

  res += " " + tokenizer.index_word[indx] # дописываем строку

  return res

In [54]:
res = buildPhrase_2("позитив добавляет годы")
res

1/1 [==============================] - 0s 101ms/step


'позитив добавляет годы понедельник'